#  **Real World Practices for Training and Regularization**

Here's a reusable **PyTorch training template** that follows **best practices in deep learning (2024+)**, including:

- ✅ `AdamW` optimizer  
- ✅ `weight_decay` (L2 regularization): `optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)`  
- ✅ `Dropout` in the model: `self.dropout = nn.Dropout(p=0.3)`  
- ✅ `Label smoothing`: `loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)`  
- ✅ `Early stopping`  
- ✅ `Data augmentation`

---


###  **1. Model Definition**
If you don't explicitly add `nn.Dropout(p)` in your model, calling `model.train()` does **nothing** in terms of dropout.

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class BestPracticeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)
```

---


###  **2. Data Loaders (with Augmentation)**

```python
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root='./data', train=True, transform=train_transform, download=True)
val_dataset = datasets.MNIST(root='./data', train=False, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
```

---

### **3. Training Template with AdamW + Early Stopping**

```python
def train_model(model, train_loader, val_loader, epochs=20, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                pred = outputs.argmax(dim=1)
                correct += pred.eq(labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct / len(val_loader.dataset)

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.4f}")

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "best_model.pt")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break
```

---


###  **4. Run the Training**

```python
model = BestPracticeNet()
train_model(model, train_loader, val_loader)
```

---

Let me know if you'd like this extended with:
- TensorBoard logging
- `torchmetrics` for F1 score, precision, recall
- Support for CIFAR10 or your custom dataset
- Integration with HuggingFace `Trainer` for even more features